<a href="https://colab.research.google.com/github/wel51x/Python-for-Data-Analysis/blob/master/ch05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 5: Getting Started with pandas
pandas will be a major tool of interest throughout much of the rest of the book. It
contains data structures and data manipulation tools designed to make data cleaning
and analysis fast and easy in Python. pandas is often used in tandem with numerical
computing tools like NumPy and SciPy, analytical libraries like statsmodels and
scikit-learn, and data visualization libraries like matplotlib. pandas adopts significant
parts of NumPy’s idiomatic style of array-based computing, especially array-based
functions and a preference for data processing without for loops.

While pandas adopts many coding idioms from NumPy, the biggest difference is that
pandas is designed for working with tabular or heterogeneous data. NumPy, by contrast,
is best suited for working with homogeneous numerical array data.

Since becoming an open source project in 2010, pandas has matured into a quite
large library that’s applicable in a broad set of real-world use cases. The developer
community has grown to over 800 distinct contributors, who’ve been helping build
the project as they’ve used it to solve their day-to-day data problems.

Throughout the rest of the book, I use the following import convention for pandas:

In [1]: import pandas as pd

Thus, whenever you see pd. in code, it’s referring to pandas. You may also find it easier
to import Series and DataFrame into the local namespace since they are so frequently
used:

In [2]: from pandas import Series, DataFrame



In [0]:
import pandas as pd

In [0]:
from pandas import Series, DataFrame

In [0]:
import numpy as np
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## 5.1 Introduction to pandas Data Structures
To get started with pandas, you will need to get comfortable with its two workhorse
data structures: Series and DataFrame. While they are not a universal solution for
every problem, they provide a solid, easy-to-use basis for most applications.

### Series

A Series is a one-dimensional array-like object containing a sequence of values (of
similar types to NumPy types) and an associated array of data labels, called its index.
The simplest Series is formed from only an array of data:

In [4]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

The string representation of a Series displayed interactively shows the index on the
left and the values on the right. Since we did not specify an index for the data, a
default one consisting of the integers 0 through N - 1 (where N is the length of the
data) is created. You can get the array representation and index object of the Series via
its values and index attributes, respectively:

In [5]:
obj.values, obj.index  # like range(4)

(array([ 4,  7, -5,  3]), RangeIndex(start=0, stop=4, step=1))

Often it will be desirable to create a Series with an index identifying each data point
with a label:

In [8]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2, sorted(obj2), obj2.index

(d    4
 b    7
 a   -5
 c    3
 dtype: int64, [-5, 3, 4, 7], Index(['d', 'b', 'a', 'c'], dtype='object'))

Compared with NumPy arrays, you can use labels in the index when selecting single
values or a set of values:

In [9]:
print(obj2['a'])
obj2['d'] = 6
obj2[['c', 'a', 'd']]

-5


c    3
a   -5
d    6
dtype: int64

Here ['c', 'a', 'd'] is interpreted as a list of indices, even though it contains
strings instead of integers.

Using NumPy functions or NumPy-like operations, such as filtering with a boolean
array, scalar multiplication, or applying math functions, will preserve the index-value
link:

In [10]:
print(obj2[obj2 > 0])
print(obj2 * 2)
print(np.exp(obj2))

d    6
b    7
c    3
dtype: int64
d    12
b    14
a   -10
c     6
dtype: int64
d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64


Another way to think about a Series is as a fixed-length, ordered dict, as it is a mapping
of index values to data values. It can be used in many contexts where you might
use a dict:

In [11]:
'b' in obj2, 'e' in obj2

(True, False)

Should you have data contained in a Python dict, you can create a Series from it by
passing the dict:

In [12]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

When you are only passing a dict, the index in the resulting Series will have the dict’s
keys in sorted order. You can override this by passing the dict keys in the order you
want them to appear in the resulting Series:

In [13]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

Here, three values found in sdata were placed in the appropriate locations, but since
no value for 'California' was found, it appears as NaN (not a number), which is considered
in pandas to mark missing or NA values. Since 'Utah' was not included in
states, it is excluded from the resulting object.

I will use the terms “missing” or “NA” interchangeably to refer to missing data. The
isnull and notnull functions in pandas should be used to detect missing data:

In [15]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [16]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

Series also has these as instance methods:

In [17]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

A useful Series feature for many applications is that it automatically aligns by index
label in arithmetic operations:

In [19]:
print(obj3)
print(obj4)
obj3 + obj4

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64


California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

If you have experience
with databases, you can think about this as being similar to a join operation.

Both the Series object itself and its index have a name attribute, which integrates with
other key areas of pandas functionality:

In [20]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

A Series’s index can be altered in-place by assignment:

In [21]:
print(obj)
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

0    4
1    7
2   -5
3    3
dtype: int64


Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### DataFrame

In [0]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

In [0]:
frame

In [0]:
frame.head()

In [0]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

In [0]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])
frame2
frame2.columns

In [0]:
frame2['state']
frame2.year

In [0]:
frame2.loc['three']

In [0]:
frame2['debt'] = 16.5
frame2
frame2['debt'] = np.arange(6.)
frame2

In [0]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

In [0]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

In [0]:
del frame2['eastern']
frame2.columns

In [0]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

In [0]:
frame3 = pd.DataFrame(pop)
frame3

In [0]:
frame3.T

In [0]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

In [0]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

In [0]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

In [0]:
frame3.values

In [0]:
frame2.values

### Index Objects

In [0]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index
index[1:]

index[1] = 'd'  # TypeError

In [0]:
labels = pd.Index(np.arange(3))
labels
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2
obj2.index is labels

In [0]:
frame3
frame3.columns
'Ohio' in frame3.columns
2003 in frame3.index

In [0]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

## Essential Functionality

### Reindexing

In [0]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

In [0]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

In [0]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3
obj3.reindex(range(6), method='ffill')

In [0]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

In [0]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

In [0]:
frame.loc[['a', 'b', 'c', 'd'], states]

### Dropping Entries from an Axis

In [0]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj
new_obj = obj.drop('c')
new_obj
obj.drop(['d', 'c'])

In [0]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [0]:
data.drop(['Colorado', 'Ohio'])

In [0]:
data.drop('two', axis=1)
data.drop(['two', 'four'], axis='columns')

In [0]:
obj.drop('c', inplace=True)
obj

### Indexing, Selection, and Filtering

In [0]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj
obj['b']
obj[1]
obj[2:4]
obj[['b', 'a', 'd']]
obj[[1, 3]]
obj[obj < 2]

In [0]:
obj['b':'c']

In [0]:
obj['b':'c'] = 5
obj

In [0]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data
data['two']
data[['three', 'one']]

In [0]:
data[:2]
data[data['three'] > 5]

In [0]:
data < 5
data[data < 5] = 0
data

#### Selection with loc and iloc

In [0]:
data.loc['Colorado', ['two', 'three']]

In [0]:
data.iloc[2, [3, 0, 1]]
data.iloc[2]
data.iloc[[1, 2], [3, 0, 1]]

In [0]:
data.loc[:'Utah', 'two']
data.iloc[:, :3][data.three > 5]

### Integer Indexes

ser = pd.Series(np.arange(3.))
ser
ser[-1]

In [0]:
ser = pd.Series(np.arange(3.))

In [0]:
ser

In [0]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

In [0]:
ser[:1]
ser.loc[:1]
ser.iloc[:1]

### Arithmetic and Data Alignment

In [0]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])
s1
s2

In [0]:
s1 + s2

In [0]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1
df2

In [0]:
df1 + df2

In [0]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})
df1
df2
df1 - df2

#### Arithmetic methods with fill values

In [0]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))
df2.loc[1, 'b'] = np.nan
df1
df2

In [0]:
df1 + df2

In [0]:
df1.add(df2, fill_value=0)

In [0]:
1 / df1
df1.rdiv(1)

In [0]:
df1.reindex(columns=df2.columns, fill_value=0)

#### Operations between DataFrame and Series

In [0]:
arr = np.arange(12.).reshape((3, 4))
arr
arr[0]
arr - arr[0]

In [0]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]
frame
series

In [0]:
frame - series

In [0]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

In [0]:
series3 = frame['d']
frame
series3
frame.sub(series3, axis='index')

### Function Application and Mapping

In [0]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame
np.abs(frame)

In [0]:
f = lambda x: x.max() - x.min()
frame.apply(f)

In [0]:
frame.apply(f, axis='columns')

In [0]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

In [0]:
format = lambda x: '%.2f' % x
frame.applymap(format)

In [0]:
frame['e'].map(format)

### Sorting and Ranking

In [0]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

In [0]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame.sort_index()
frame.sort_index(axis=1)

In [0]:
frame.sort_index(axis=1, ascending=False)

In [0]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

In [0]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

In [0]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame
frame.sort_values(by='b')

In [0]:
frame.sort_values(by=['a', 'b'])

In [0]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

In [0]:
obj.rank(method='first')

In [0]:
# Assign tie values the maximum rank in the group
obj.rank(ascending=False, method='max')

In [0]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})
frame
frame.rank(axis='columns')

### Axis Indexes with Duplicate Labels

In [0]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

In [0]:
obj.index.is_unique

In [0]:
obj['a']
obj['c']

In [0]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df
df.loc['b']

## Summarizing and Computing Descriptive Statistics

In [0]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

In [0]:
df.sum()

In [0]:
df.sum(axis='columns')

In [0]:
df.mean(axis='columns', skipna=False)

In [0]:
df.idxmax()

In [0]:
df.cumsum()

In [0]:
df.describe()

In [0]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

### Correlation and Covariance

conda install pandas-datareader

In [0]:
price = pd.read_pickle('examples/yahoo_price.pkl')
volume = pd.read_pickle('examples/yahoo_volume.pkl')

import pandas_datareader.data as web
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})

In [0]:
returns = price.pct_change()
returns.tail()

In [0]:
returns['MSFT'].corr(returns['IBM'])
returns['MSFT'].cov(returns['IBM'])

In [0]:
returns.MSFT.corr(returns.IBM)

In [0]:
returns.corr()
returns.cov()

In [0]:
returns.corrwith(returns.IBM)

In [0]:
returns.corrwith(volume)

### Unique Values, Value Counts, and Membership

In [0]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [0]:
uniques = obj.unique()
uniques

In [0]:
obj.value_counts()

In [0]:
pd.value_counts(obj.values, sort=False)

In [0]:
obj
mask = obj.isin(['b', 'c'])
mask
obj[mask]

In [0]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match)

In [0]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
data

In [0]:
result = data.apply(pd.value_counts).fillna(0)
result

## Conclusion

In [0]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS